Cython,
Nikita Osipov,

Данный файл представляет собой мини-набор сравнений между функциями, написанными на языке Python, и функциями, оптимизированными за счет Cython. Для тестов были использованы алгоритмы определения простоты числа (miller-rabin and ibn al banna). 
Первым дейcтвием является загрузка Cython. (Cython устанавливается вместе с Anaconda по умолчанию, но если использовать браузерную версию https://jupyter.org/try, то это необходимо).

In [23]:
conda install cython

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


Чтобы использовать Cython в блокноте Jupyter, нам сначала нужно импортировать расширение Cython Jupyter:

In [4]:
%load_ext cython

Проведем тест функции, без использования возможностей Cython.

In [5]:
#%%cython -a
import random    
import math                     
            
def miller_rabin(num,rnds):
    if num == 2 or num == 3 or num == 5 or num == 7 or num == 11:
        return True     
    elif num == 4 or num == 6 or num == 8 or num == 9 or num == 10:
        return False      
    elif num > 2 and rnds > 0:      
        if num % 2 == 0:
            return False            
        else:
            s = 0
            t = num - 1           
            while t % 2 == 0:
                t //= 2
                s +=1           
            for i in range(rnds):
                a = random.randint(2,num-2)
                x = pow(a,t,num)
                if x == 1 or x == num - 1:
                    continue               
                for j in range(s-1):
                    x = pow(x,2,num)
                    if x == num - 1:
                        break
                else:
                    return False
            return True            
    else:
        print('Неверные данные\nПервый аргумент >= 2\nВторой > 0')

Узнем скорость работы.

In [13]:
ex1 = 703464731
%timeit -n 50 miller_rabin(ex1,1000)

5.32 ms ± 214 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


Теперь добавим информацию о типах данных.

In [14]:
%%cython -a
import random    
import math                     
            
def miller_rabin(long int num, int rnds):
    cdef int t,s,i,a,x,j
    if num == 2 or num == 3 or num == 5 or num == 7 or num == 11:
        return True     
    elif num == 4 or num == 6 or num == 8 or num == 9 or num == 10:
        return False      
    elif num > 2 and rnds > 0:      
        if num % 2 == 0:
            return False            
        else:
            s = 0
            t = num - 1           
            while t % 2 == 0:
                t //= 2
                s +=1           
            for i in range(rnds):
                a = random.randint(2,num-2)
                x = pow(a,t,num)
                if x == 1 or x == num - 1:
                    continue               
                for j in range(s-1):
                    x = pow(x,2,num)
                    if x == num - 1:
                        break
                else:
                    return False
            return True            
    else:
        print('Неверные данные\nПервый аргумент >= 2\nВторой > 0')

Узнаем скорость работы.

In [15]:
%timeit -n 50 miller_rabin(ex1,1000)

5.03 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


На примере алгоритма Миллера-Рабина мы не видим значительных изменений в скорости. Перейдем к следующему алгоритму.

Данный алгоритм (Ibn al banna) или перебор всех делителей до корня из числа.

In [16]:
import math

def ibn_al_banna(x):
    k = 0 
    s = math.floor(math.sqrt(x))
    if x == 2:
        return True
    elif x % 2 == 0:
        return False
    elif x == 1:
        return False
    else:
        for i in range(2,s+1):
            if x % i == 0:
                k = 1
                return False
                break                              
            else:
                continue
    if k == 0:
        return True

Узнаем скорость работы функции:

In [17]:
%timeit -n 50 ibn_al_banna(ex1)

2.06 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


Теперь добавим информацию о типах данных.

In [20]:
%%cython -a
import math

def ibn_al_banna(long int x):
    cdef int k,s,i
    k = 0 
    s = math.floor(math.sqrt(x))
    if x == 2:
        return True
    elif x % 2 == 0:
        return False
    elif x == 1:
        return False
    else:
        for i in range(2,s+1):
            if x % i == 0:
                k = 1
                return False
                break                              
            else:
                continue
    if k == 0:
        return True

Проведем заключительные замеры. 

In [21]:
%timeit -n 50 ibn_al_banna(ex1)

362 µs ± 67.3 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


Данное значение значительно меньше, что говорит об ускорении работы функции. 
Если говорить более точно, данная функция с Cython работает в 6 раз быстрее.